In [265]:
# Run and collapse this cell.

import csv
import numpy
import pprint
import requests
from dataclasses import dataclass
from scipy import optimize

@dataclass
class Equipment:
    # Generic representation of anything that can go on your character.
    name_en: str
    rarity: str
    type: str
    defense: int
    skills: ((str, int), (str, int)) # may be (empty string, 0)
    set_bonus_skills: ((str, int), (str, int)) # may be (empty string, 0).
    deco_balance: (int, int, int, int) # may be negative, when the equipment is a deco.

    def get_skill_level(self, skill):
        if self.skills[0][0] == skill:
            return self.skills[0][1]
        if self.skills[1][0] == skill:
            return self.skills[1][1]
        return 0
    
    def get_skill_contribution(self, skill):
        base = self.get_skill_level(skill)
        if self.set_bonus_skills[0][0] == skill:
            return base + 1.0 / self.set_bonus_skills[0][1]
        if self.set_bonus_skills[1][0] == skill:
            return base + 1.0 / self.set_bonus_skills[1][1]
        return base

In [266]:
# Run and collapse this cell. EDIT FOR WEAPON DECO SLOTS!
# Weapon deco is at the very bottom.

def build_equipment_table():
    def get_data(local, remote) -> csv.DictReader:
        try:
            with open(local, encoding="utf8") as f:
                return csv.DictReader(f.read().split("\n"))
        except OSError:
            return csv.DictReader(
                requests.get(remote)
                .text.split("\n")
            )
        assert False, "this should never happen"

    armor_table = list(get_data(
            "mhw_data/armor_base.csv",
            "https://raw.githubusercontent.com/gatheringhallstudios/MHWorldData/master/source_data/armors/armor_base.csv"
        ))

    armor_skills_table = dict((x["base_name_en"], x) for x in get_data(
            "mhw_data/armor_skills_ext.csv",
            "https://raw.githubusercontent.com/gatheringhallstudios/MHWorldData/master/source_data/armors/armor_skills_ext.csv"
        ))

    armor_sets_table = list(x for x in get_data(
            "mhw_data/armorset_base.csv",
            "https://raw.githubusercontent.com/gatheringhallstudios/MHWorldData/master/source_data/armors/armorset_base.csv"
        ))

    armor_set_bonus_table = dict(
            (x["name_en"], x)
            for x in get_data(
                "mhw_data/armorset_bonus_base.csv",
                "https://raw.githubusercontent.com/gatheringhallstudios/MHWorldData/master/source_data/armors/armorset_bonus_base.csv"
        ))

    charm_table = list(get_data(
            "mhw_data/charm_base.csv",
            "https://raw.githubusercontent.com/gatheringhallstudios/MHWorldData/master/source_data/charms/charm_base.csv"
        ))

    deco_table = list(get_data(
            "mhw_data/decoration_base.csv",
            "https://raw.githubusercontent.com/gatheringhallstudios/MHWorldData/master/source_data/decorations/decoration_base.csv"
        ))

    # join and reformat.

    equipment_table = []
    for armor in armor_table:
        skill_ext = armor_skills_table[armor["name_en"]]

        maybe_armor_set = tuple(
                armor_set["bonus"] for armor_set in armor_sets_table
                if armor["name_en"] in (armor_set["head"], armor_set["chest"], armor_set["arms"], armor_set["waist"], armor_set["legs"])
            )
        maybe_armor_set_bonus = armor_set_bonus_table[maybe_armor_set[0]] if maybe_armor_set and maybe_armor_set[0] in armor_set_bonus_table else None

        equipment_table.append(Equipment(
            name_en = armor["name_en"],
            rarity = int(armor["rarity"]),
            type = armor["type"],
            defense = armor["defense_base"],
            skills = (
                    (skill_ext["skill1_name"], int(skill_ext["skill1_level"]) if skill_ext["skill1_name"] != "" else 0),
                    (skill_ext["skill2_name"], int(skill_ext["skill2_level"]) if skill_ext["skill2_name"] != "" else 0)
                ),
            set_bonus_skills = (
                    (f"{maybe_armor_set_bonus['skill1_name']} ({maybe_armor_set[0]})", int(maybe_armor_set_bonus["skill1_required"])) if (maybe_armor_set_bonus and maybe_armor_set_bonus["skill1_name"] != "") else ("", 0),
                    (f"{maybe_armor_set_bonus['skill2_name']} ({maybe_armor_set[0]})", int(maybe_armor_set_bonus["skill2_required"])) if (maybe_armor_set_bonus and maybe_armor_set_bonus["skill2_name"] != "") else ("", 0)
                ),
            deco_balance = (
                    int(int(armor["slot_1"]) == 1) + int(int(armor["slot_2"]) == 1) + int(int(armor["slot_3"]) == 1),
                    int(int(armor["slot_1"]) == 2) + int(int(armor["slot_2"]) == 2) + int(int(armor["slot_3"]) == 2),
                    int(int(armor["slot_1"]) == 3) + int(int(armor["slot_2"]) == 3) + int(int(armor["slot_3"]) == 3),
                    int(int(armor["slot_1"]) == 4) + int(int(armor["slot_2"]) == 4) + int(int(armor["slot_3"]) == 4)
                ),
        ))
    for charm in charm_table:
        equipment_table.append(Equipment(
            name_en = charm["name_en"],
            rarity = int(charm["rarity"]),
            type = "charm",
            defense = 0,
            skills = (
                    (charm["skill1_name"], int(charm["skill1_level"]) if charm["skill1_name"] != "" else 0),
                    (charm["skill2_name"], int(charm["skill2_level"]) if charm["skill2_name"] != "" else 0),
                ),
            set_bonus_skills = (("", 0), ("", 0)),
            deco_balance = (0, 0, 0, 0),
        ))
    for deco in deco_table:
        equipment_table.append(Equipment(
            name_en = deco["name_en"],
            rarity = int(deco["rarity"]),
            type = "deco",
            defense = 0,
            skills = (
                    (deco["skill1_name"], int(deco["skill1_level"]) if deco["skill1_name"] != "" else 0),
                    (deco["skill2_name"], int(deco["skill2_level"]) if deco["skill2_name"] != "" else 0),
                ),
            set_bonus_skills = (("", 0), ("", 0)),
            deco_balance = (
                    -1 * int(int(deco["slot"]) == 1),
                    -1 * int(int(deco["slot"]) == 2),
                    -1 * int(int(deco["slot"]) == 3),
                    -1 * int(int(deco["slot"]) == 4)
                ),
        ))
    equipment_table.append(Equipment(
        name_en = "Weapon (Remember to adjust slots!)",
        rarity = 0,
        type = "weapon",
        defense = 0,
        skills = (
                ("", 0),
                ("", 0),
            ),
        set_bonus_skills = (("", 0), ("", 0)),
        deco_balance = (0, 0, 1, 1),
    ))
    return equipment_table

EQUIPMENT_TABLE = tuple(build_equipment_table())

SKILL_LIST = tuple(sorted(
        {equipment.skills[0][0] for equipment in EQUIPMENT_TABLE}
            .union({equipment.skills[1][0] for equipment in EQUIPMENT_TABLE})
            .union({equipment.set_bonus_skills[0][0] for equipment in EQUIPMENT_TABLE})
            .union({equipment.set_bonus_skills[1][0] for equipment in EQUIPMENT_TABLE})
            .difference({""})
    ))

DECO_LIST = tuple(sorted(set(equipment.name_en for equipment in EQUIPMENT_TABLE if equipment.type == "deco")))

In [267]:
# Run and collapse this cell.

SKILL_MATRIX = numpy.vstack(
    tuple(
        numpy.fromiter((equipment.get_skill_contribution(skill) for equipment in EQUIPMENT_TABLE), float)
        for skill in SKILL_LIST
    )
)

def build_fixed_constraints():
    # A set cannot contain more than one of these.
    head_row = numpy.fromiter((equip.type == "head" for equip in EQUIPMENT_TABLE), int)
    chest_row = numpy.fromiter((equip.type == "chest" for equip in EQUIPMENT_TABLE), int)
    arms_row = numpy.fromiter((equip.type == "arms" for equip in EQUIPMENT_TABLE), int)
    waist_row = numpy.fromiter((equip.type == "waist" for equip in EQUIPMENT_TABLE), int)
    legs_row = numpy.fromiter((equip.type == "legs" for equip in EQUIPMENT_TABLE), int)
    charm_row = numpy.fromiter((equip.type == "charm" for equip in EQUIPMENT_TABLE), int)
    weapon_row = numpy.fromiter((equip.type == "weapon" for equip in EQUIPMENT_TABLE), int)
    equipment_matrix = numpy.vstack((head_row, chest_row, arms_row, waist_row, legs_row, charm_row, weapon_row))
    equipment_upper_bound = numpy.ones(7)

    # Converts slots to "balance". Deco "balance" represents the opportunity to use a deco.
    # Using a n slot deco means you lose the ability to fit in a different 1..n slot deco.
    # Balance for each slot should be nonnegative.
    unused_4_slots = numpy.fromiter((equip.deco_balance[3] for equip in EQUIPMENT_TABLE), int)
    unused_3_slots = numpy.fromiter((equip.deco_balance[2] for equip in EQUIPMENT_TABLE), int)
    unused_2_slots = numpy.fromiter((equip.deco_balance[1] for equip in EQUIPMENT_TABLE), int)
    unused_1_slots = numpy.fromiter((equip.deco_balance[0] for equip in EQUIPMENT_TABLE), int)
    deco_unused_matrix = numpy.vstack((unused_4_slots, unused_3_slots, unused_2_slots, unused_1_slots))
    deco_balance_matrix = numpy.tri(4) @ deco_unused_matrix
    deco_balance_lower_bound = numpy.zeros(4)

    return (
            numpy.concatenate((equipment_matrix, -deco_balance_matrix)),
            numpy.concatenate((equipment_upper_bound, -deco_balance_lower_bound)),
        )

FIXED_CONSTRAINTS_MATRIX, FIXED_CONSTRAINTS_UB = build_fixed_constraints()  


# The actual configs
The ones you are most likely to edit are placed further below to reduce scrolling.

In [268]:
# By substring, to filter out entire sets, or specific pieces.
user_excluded_substrings_en = {
    "Dragonhead", "Dragonhide", "Dragonclaws", "Dragonbarbs", "Dragonfeet", # Fatalis 
    "Escadora", # Alatreon
    "γ", # Arch-Tempered
}

assert all(any(equip.name_en.find(excluded) != -1 for equip in EQUIPMENT_TABLE) for excluded in user_excluded_substrings_en), next(excluded for excluded in user_excluded_substrings_en if all(equip.name_en.find(excluded) == -1 for equip in EQUIPMENT_TABLE)) + " does not match any armor"
pprint.pprint(dict(
        (excluded, list(equip.name_en for equip in EQUIPMENT_TABLE if equip.name_en.find(excluded) != -1))
        for excluded in user_excluded_substrings_en 
    ))

{'Dragonbarbs': ['Dragonbarbs α+', 'Dragonbarbs β+'],
 'Dragonclaws': ['Dragonclaws α+', 'Dragonclaws β+'],
 'Dragonfeet': ['Dragonfeet α+', 'Dragonfeet β+'],
 'Dragonhead': ['Dragonhead α+', 'Dragonhead β+'],
 'Dragonhide': ['Dragonhide α+', 'Dragonhide β+'],
 'Escadora': ['Escadora Wisdom α+',
              'Escadora Soul α+',
              'Escadora Armguards α+',
              'Escadora Might α+',
              'Escadora Sheath α+',
              'Escadora Wisdom β+',
              'Escadora Soul β+',
              'Escadora Armguards β+',
              'Escadora Might β+',
              'Escadora Sheath β+'],
 'γ': ['Vaal Hazak Helm γ',
       'Vaal Hazak Mail γ',
       'Vaal Hazak Braces γ',
       'Vaal Hazak Coil γ',
       'Vaal Hazak Greaves γ',
       'Kirin Horn γ',
       'Kirin Jacket γ',
       'Kirin Longarms γ',
       'Kirin Hoop γ',
       'Kirin Leg Guards γ',
       'Kaiser Crown γ',
       'Kaiser Mail γ',
       'Kaiser Vambraces γ',
       'Kaiser Coil γ',
    

In [269]:
# For reference, uncomment. You can probably read this from the equipment box though.
# print(DECO_LIST)
# You can update this as you play. You don't need to add literally every deco, just the important ones or ones that are nice to have.
user_maximum_decos = {
    "Steadfast Jewel+ 4": 1,
    "Miasma Jewel+ 4": 1,
    "Expert Jewel+ 4": 1,
    "Heavy Artillery Jewel+ 4": 1, # for fatalis??
    "Wind Resist/Protection Jewel 4": 1,
    "Brace/Vitality Jewel 4": 1,
    "Draw/Medicine Jewel 4": 1,
    "Fortitude/Attack Jewel 4": 1,
    "Crisis/Attack Jewel 4": 1,
    "Guardian/Attack Jewel 4": 1,
    "Bomber/Attack Jewel 4": 1,
    "Flight/Expert Jewel 4": 1,
    "Sheath/Expert Jewel 4": 1,
    "Crisis/Expert Jewel 4": 1,
    "Stonethrower/Expert Jewel 4": 1,
    "Resistor/Handicraft Jewel 4": 1,
    "Jumping/Physique Jewel 4": 1,
    "Friendship/Physique Jewel 4": 1,
    "Footing/Evasion Jewel 4": 1,
    "Slider/Evasion Jewel 4": 1,
    "Crisis/Evasion Jewel 4": 1,
    "Stonethrower/Evasion Jewel 4": 2,
    "Draw/Maintenance Jewel 4": 1,
    "Enhancer/Maintenance Jewel 4": 1,
    "Challenger/Maintenance Jewel 4": 1,
    "Flawless/Maintenance Jewel 4": 1,

    "Earplug Jewel 3": 5,
    "Brace Jewel 3": 5,
    "Handicraft Jewel 3": 2,
    "Shaver Jewel 3": 1,
}

# Conveniences if you have mostly everything or are willing to find them 
user_maximum_decos.update(dict(
        (deco, 1000) for deco in DECO_LIST if deco.endswith("1")
    ))
user_maximum_decos.update(dict(
        (deco, 1000) for deco in DECO_LIST if deco.endswith("2")
    ))
# user_maximum_decos.update(dict(
#         (deco, 1000) for deco in DECO_LIST if deco.endswith("3")
#     ))
# user_maximum_decos.update(dict(
#         (deco, 1000) for deco in DECO_LIST if deco.endswith("4")
#     ))

assert all(owned in DECO_LIST for owned in user_maximum_decos), next(owned for owned in user_maximum_decos if owned not in DECO_LIST)

In [270]:
# This is placed as low as possible to reduce scrolling up from output below.
# Reminder to put any relevant decos up top!
user_minimum_skills = {
    "Flinch Free": 1,

    "Artillery Secret (Brachydios Will)": 1,
    "Artillery": 5,
    "Capacity Boost": 1,

    "Evade Extender": 3,
    "Stun Resistance": 3,
    "Quick Sheath": 3,
    "Health Boost": 3,
    "Guard Up": 1,
    "Guard": 3,
    
    "Razor Sharp/Spare Shot": 1,
    "Clutch Claw Boost": 1,

    "Blast Attack": 3, # pin my current raging brachy chest
}

assert all(skill in SKILL_LIST for skill in user_minimum_skills), next(skill for skill in user_minimum_skills if skill not in SKILL_LIST)

In [271]:
# Run and collapse this cell.

# Set must contain the user set skills at set levels.
def build_user_constraints():    
    if len(user_minimum_skills) > 0:
        skill_indicators = numpy.vstack(
                tuple(
                    numpy.fromiter((selected == skill for skill in SKILL_LIST), int)
                    for selected in sorted(user_minimum_skills)
                )
            )
        user_skill_matrix = skill_indicators @ SKILL_MATRIX
        user_skill_lower_bound = numpy.fromiter((level for skill, level in sorted(user_minimum_skills.items())), int)
    else:
        user_skill_matrix = numpy.zeros((0, len(EQUIPMENT_TABLE)))
        user_skill_lower_bound = numpy.zeros(0)

    # Set must not contain more decos than owned.
    user_deco_matrix = numpy.vstack(
            tuple(
                numpy.fromiter((equip.name_en == deco for equip in EQUIPMENT_TABLE), int)
                for deco in DECO_LIST
            )
        )
    user_deco_upper_bound = numpy.fromiter((user_maximum_decos[deco] if deco in user_maximum_decos else 0 for deco in DECO_LIST), int)

    # Set must not contain excluded equipment. (Their counts each must be zero, so their sum must be zero.)
    user_exclude_matrix = numpy.asmatrix(numpy.fromiter((any(equip.name_en.find(excluded) != -1 for excluded in user_excluded_substrings_en) for equip in EQUIPMENT_TABLE), int))
    user_exclude_upper_bound = numpy.zeros(1)

    return (
            numpy.concatenate((-user_skill_matrix, user_deco_matrix, user_exclude_matrix)),
            numpy.concatenate((-user_skill_lower_bound, user_deco_upper_bound, user_exclude_upper_bound)),
        )

user_constraints_matrix, user_constraints_ub = build_user_constraints()


In [272]:
# Maximizing defense:
DEFENSE_ROW = numpy.fromiter((equip.defense for equip in EQUIPMENT_TABLE), int)

def interpret(res, integrality = True):
    if res.status != 0:
        print("No solution :(")
        return

    selector = numpy.fromiter((round(count) for count in res.x), int) if integrality else res.x

    for count, equip in zip(selector, EQUIPMENT_TABLE):
        if count > 0:
            print(count, equip.name_en, end=" ")
            if equip.skills[0][1] != 0:
                print(equip.skills[0], end=" ")
            if equip.skills[1][1] != 0:
                print(equip.skills[1], end=" ")
            print()
    
    for count, skill in zip(SKILL_MATRIX @ selector, SKILL_LIST):
        if count > 0:
            print(skill, count) 

    print(f"defense: {DEFENSE_ROW @ selector}")

interpret(optimize.linprog(
        c=-DEFENSE_ROW,
        A_ub=numpy.concatenate((FIXED_CONSTRAINTS_MATRIX, user_constraints_matrix)),
        b_ub=numpy.concatenate((FIXED_CONSTRAINTS_UB, user_constraints_ub)),
        bounds=(0, 7),
        integrality=1,
    ))

1 Grand God's Peer Feet β+ ('Health Boost', 3) 
1 Brachydium Helm α+ ('Weakness Exploit', 1) ('Guard', 2) 
1 Brachydium Mail α+ ('Agitator', 2) ('Blast Attack', 3) 
1 Brachydium Braces α+ ('Agitator', 2) ('Artillery', 3) 
1 Brachydium Faulds α+ ('Agitator', 3) ('Artillery', 2) 
1 Razor Sharp Charm ('Razor Sharp/Spare Shot', 1) 
2 Jumping Jewel 2 ('Evade Extender', 1) 
3 Sheath Jewel 1 ('Quick Sheath', 1) 
1 Steadfast Jewel 1 ('Stun Resistance', 1) 
1 Shield Jewel 2 ('Guard Up', 1) 
1 Ironwall Jewel 1 ('Guard', 1) 
1 Magazine Jewel 2 ('Capacity Boost', 1) 
1 Steadfast Jewel+ 4 ('Stun Resistance', 2) 
1 Jumping/Physique Jewel 4 ('Evade Extender', 1) ('Constitution', 1) 
1 Brace/Vitality Jewel 4 ('Flinch Free', 1) ('Health Boost', 1) 
1 Shaver Jewel 3 ('Clutch Claw Boost', 1) 
1 Weapon (Remember to adjust slots!) 
Agitator 7.0
Agitator Secret (Brachydios Will) 2.0
Artillery 5.0
Artillery Secret (Brachydios Will) 1.0
Blast Attack 3.0
Capacity Boost 1.0
Clutch Claw Boost 1.0
Constitution 1.

In [273]:
# Lists the maximum possible level for each skill.
def maximize_skill(maximized_skill):
    maximized_skill_indicator = numpy.fromiter((skill == maximized_skill for skill in SKILL_LIST), int)

    return optimize.linprog(
        # Alternate view of product: maximize equipment with skill. higher skill values prioritized
        c=-maximized_skill_indicator @ SKILL_MATRIX,
        A_ub=numpy.concatenate((FIXED_CONSTRAINTS_MATRIX, user_constraints_matrix)),
        b_ub=numpy.concatenate((FIXED_CONSTRAINTS_UB, user_constraints_ub)),
        bounds=(0, 7),
        integrality=1, # do not set to zero!
    )

def list_supersets():
    letter = None
    for maximized_skill in SKILL_LIST:
        res = maximize_skill(maximized_skill)
        if res.status == 0 and round(-res.fun) != 0:
            if maximized_skill[0] != letter:
                if letter != None:
                    print()
                letter = maximized_skill[0]
            print((maximized_skill, round(-res.fun)), end=", ")

list_supersets()

('Agitator', 7), ('Agitator Secret (Brachydios Will)', 2), ('Artillery', 5), ('Artillery Secret (Brachydios Will)', 1), 
('Blast Attack', 5), ('Blast Resistance', 1), 
('Capacity Boost', 1), ('Clutch Claw Boost', 1), ('Constitution', 1), ('Critical Draw', 1), ('Critical Eye', 1), 
('Earplugs', 1), ('Evade Extender', 3), 
('Flinch Free', 1), ('Focus', 1), 
('Guard', 3), ('Guard Up', 1), 
('Health Boost', 4), ('Heroics', 2), 
('Punishing Draw (Frostfang Absolute Art)', 1), 
('Quick Sheath', 3), 
('Razor Sharp/Spare Shot', 1), ('Recovery Up', 1), 
('Stun Resistance', 4), 
('Weakness Exploit', 3), 

## Niche usage below

In [274]:
# # Generates a random set.
# import random
# # random.seed(0)
# interpret(optimize.linprog(
#         c=-numpy.fromiter((random.random() for _ in EQUIPMENT_TABLE), float), # random weights
#         A_ub=numpy.concatenate((FIXED_CONSTRAINTS_MATRIX, user_constraints_matrix)),
#         b_ub=numpy.concatenate((FIXED_CONSTRAINTS_UB, user_constraints_ub)),
#         bounds=(0, 7),
#         integrality=1,
#     ))

In [275]:
# # Minimizes rarity (for earlygame?)

# interpret(optimize.linprog(
#         c=numpy.fromiter((equip.rarity for equip in EQUIPMENT_TABLE), float) + DEFENSE_ROW / 1000,
#         A_ub=numpy.concatenate((FIXED_CONSTRAINTS_MATRIX, user_constraints_matrix)),
#         b_ub=numpy.concatenate((FIXED_CONSTRAINTS_UB, user_constraints_ub)),
#         bounds=(0, 7),
#         integrality=1,
#     ))

In [276]:
# # Maximize skills with weights. Will exceed limits so adjust weights until reasonable.
# nice_to_have_skill_weights = {
#     "Guard": 1,
# }
# nice_to_have_vec = numpy.fromiter((nice_to_have_skill_weights[skill] if skill in nice_to_have_skill_weights else 0 for skill in SKILL_LIST), int)

# interpret(optimize.linprog(
#         c=-nice_to_have_vec @ SKILL_MATRIX,
#         A_ub=numpy.concatenate((FIXED_CONSTRAINTS_MATRIX, user_constraints_matrix)),
#         b_ub=numpy.concatenate((FIXED_CONSTRAINTS_UB, user_constraints_ub)),
#         bounds=(0, 7),
#         integrality=1,
#     ))